In [2]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data, BatchRetrySettings
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential, ManagedIdentityCredential, AzureCliCredential, InteractiveBrowserCredential

from pathlib import Path
import pandas as pd
import glob
from dotenv import load_dotenv
import os

In [3]:
load_dotenv(Path("..", "..", ".env"))

True

In [4]:
# Service Principle is required - automatically done if env variables are available (see .env file) ->
# # AZURE_TENANT_ID, AZURE_CLIENT_SECRET, AZURE_OBJECT_ID
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [5]:
ml_client = MLClient.from_config(
    credential=credential, file_name=str(Path("..", "..", "config.json")) # AzureCliCredential()
)

Found the config file in: ./../../config.json


In [6]:
iris_data_unlabeled = ml_client.data.get(name="template_data_iris_unlabeled", label="latest") # datastore path to be used!!

In [7]:
job = ml_client.batch_endpoints.invoke(
   endpoint_name=os.getenv("BATCH_ENDPOINT_NAME"),
   input=Input(type=AssetTypes.URI_FOLDER, path=iris_data_unlabeled.path),
)

In [12]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
batch-endpoint-pw2907,batchjob-c221238e-7505-46df-9c8b-7cd2b791c96b,pipeline,Completed,Link to Azure Machine Learning studio


In [13]:
ml_client.jobs.download(name=job.name, output_name='score', download_path='./') # somehow does not work

In [14]:

output_files = glob.glob("./score/*.parquet")
score = pd.concat((pd.read_parquet(f) for f in output_files))

ValueError: No objects to concatenate

In [30]:
output_files

[]